In [ ]:
import os
import numpy as np
from functions_InverseSolutions import applyBeamformer, showResult, prepareInverseSolution_group
%matplotlib inline

import mne
from mayavi import mlab

mlab.init_notebook('png')
mne.viz.set_3d_backend('mayavi')

sensors = 'mag'
tmin, tmax = -0.1, 0.6

mainFolder = "..\Data\\"
print('main folder folder: ', mainFolder)

meg_MainFolder = mainFolder + "MEG_Data\Data="
print('meg main folder folder: ', meg_MainFolder)

dataFolder = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', dataFolder)

source_MainFolder = "..\SourceLocalization\SourceEstimates\Data="
sourceFolder = source_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Source folder: ', sourceFolder)

subjects_dir = '..\SourceLocalization\subjects\\'
print('Subjects directory: ', subjects_dir)

forwardModelsFolder = '..\SourceLocalization\ForwardModels\\'
print('Forward models folder: ', forwardModelsFolder)

spatialFiltersFolder = '..\SourceLocalization\SpatialFilters\\Data='
spatialFiltersFolder = spatialFiltersFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Spatial filters folder: ', spatialFiltersFolder)

statResultsFolder  = '..\SourceLocalization\Results\\Data='
statResultsFolder = statResultsFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Statisctics results folder: ', statResultsFolder)

classifiers_MainFolder = "..\Classifiers\Data="    
clsfFolder = classifiers_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Classifiers folder: ', clsfFolder)    

# Pick the classifiers based on the their training window
peak_indices = [20, 50]
    
print('Peak indices: ', peak_indices)
    
# Task name for the classifiers
task_name = 'all_predLevel'
print('Task name: ', task_name)
ids_filename = 'Participant_ids.txt'
participants_filename = 'Participants.txt'

# All subjects
s_id_list_all = np.loadtxt(mainFolder+ids_filename, dtype=str)
s_id_list_all = s_id_list_all.tolist()


participant_names = np.loadtxt(mainFolder+participants_filename, dtype=str)
participant_names = participant_names.tolist()


print('Number of subjects: ', len(participant_names))

if os.path.exists(subjects_dir + '\\' + participant_names[0] + '\surf\\'):  
    print('exists!')


In [ ]:
# Read fsaverage
src_ave = mne.read_source_spaces(subjects_dir + 'fsaverage\\bem\\fsaverage-ico-5-src.fif')



#### Choose the conditions for contrasting

conditions = ['living_real_8', 'living_real_9', 'living_real_10', 'object_real_8', 'object_real_9', 'object_real_10']
condition_names =  ["real"]

n_subjects = len(s_id_list_all)
print('number of subjects: ', n_subjects)

print('conditions: \n', conditions)


### Define time limits for inverse solutins

tminData, tmaxData = 0.1, 0.4  #0.1, 0.4 #-0.04, 0
print('tmin for data: ', tminData)
print('tmax for data: ', tmaxData)
tminNoiseCov, tmaxNoiseCov = -0.1, -0.05
print('tmin for noise covariance: ', tminNoiseCov)
print('tmax for noise covariance: ', tmaxNoiseCov)
tminEpoch = -0.1
print('tmin for generated epochs: ', tminEpoch)
smoothAmount = 70
print('Smoothing amount: ', smoothAmount)
inv_sol_method = 'beamformer'
print('Inverse solution method: ', inv_sol_method)

### Use beamformer for computing source estimates

In [ ]:
print('======= Applying Beamformer ========')

stc_real_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'

if os.path.exists(statResultsFolder + stc_real_filename) != True:  # Compute the source estimate

    stc_fsave_all_real, n_times, tstep = applyBeamformer(conditions, s_id_list_all, 
                                                         n_subjects, participant_names, tminData, tmaxData, 
                                                         tminNoiseCov, tmaxNoiseCov, tminEpoch, smoothAmount, task_name,
                                                         clsfFolder, peak_indices, src_ave, spatialFiltersFolder,
                                                         subjects_dir, dataFolder, sensors, forwardModelsFolder)

    print('stc is saved in ', stc_real_filename)
    np.save(statResultsFolder + stc_real_filename, stc_fsave_all_real)

    # Check if there is 0
    if len(np.where(stc_fsave_all_real == 0)[0]) == 0:
        print('No Zeros!')
    else:
        print('There are zeros at ', np.where(stc_fsave_all_real == 0)[0])

else:
    tstep=0.01
    # time period of interest
    tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4
    stc_real_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'
    print('Source estimate exists!\n Loading from file'+statResultsFolder+stc_real_filename+'...')
    stc_fsave_all_real_clfRange = np.load(statResultsFolder+stc_real_filename)
    print('shape of real sounds source estimates: ', stc_fsave_all_real_clfRange.shape)
    print('There are zeros at ', np.where(stc_fsave_all_real_clfRange == 0))
    
    # Baseline
    tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0
    stc_baseline_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
    print('Source estimate exists!\n Loading from file'+stc_baseline_filename+'...')
    print(statResultsFolder+stc_baseline_filename)
    stc_fsave_all_real_baseline = np.load(statResultsFolder+stc_baseline_filename)
    print('shape of real sounds source estimates: ', stc_fsave_all_real_baseline.shape)
    print('There are zeros at ',np.where(stc_fsave_all_real_baseline == 0))


### Check data before doing anything!!

print('Baseline: \n', stc_fsave_all_real_baseline)

print('Clf range: \n', stc_fsave_all_real_clfRange)

### Take mean 
# take mean over participants (dimension 2)
stc_fsave_all_real_clfRange_avg = np.mean(stc_fsave_all_real_clfRange, axis=2)
print('Shape stc_fsave_all_real_clfRange_avg after avg over subjects: ', stc_fsave_all_real_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_clfRange_avg= np.mean(stc_fsave_all_real_clfRange_avg, axis=1)
print('Shape stc_fsave_all_real_clfRange_avg after avg across time: ', stc_fsave_all_real_clfRange_avg.shape)
print(stc_fsave_all_real_clfRange_avg.shape)

print('stc_fsave_all_real_clfRange_avg: ', stc_fsave_all_real_clfRange_avg)

# take mean over participants (dimension 2)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline, axis=2)
print('Shape stc_fsave_all_real_baseline_avg after avg over subjects: ', stc_fsave_all_real_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline_avg, axis=1)
print('Shape stc_fsave_all_real_baseline_avg after avg across time: ', stc_fsave_all_real_baseline_avg.shape)
stc_fsave_all_real_baseline_avg.shape

print('stc_fsave_all_real_baseline_avg: ', stc_fsave_all_real_baseline_avg)

### Generate stc group
stc_cls = prepareInverseSolution_group(stc_fsave_all_real_clfRange_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_baseline = prepareInverseSolution_group(stc_fsave_all_real_baseline_avg, subjects_dir, tstep=tstep,
                                            tmin_tmp=tminData_baseline_tmp)


### Plot

'real_baseline_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount)

showResult('fsaverage', sourceFolder, stc_cls,
           'real_clf_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount), subjects_dir,
           minimum=1300000, maximum=3000000)

showResult('fsaverage', sourceFolder, stc_baseline,
           'real_baseline_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount), subjects_dir,
           minimum=99000, maximum=440000)

# Compute relative change from baseline
stc_fsave_all_real_diff_avg = 100*(stc_fsave_all_real_clfRange_avg-stc_fsave_all_real_baseline_avg)/stc_fsave_all_real_baseline_avg




#print(np.min(stc_fsave_all_real_diff_avg))

# Threshold for plotting
thresh= 700 #550
print('Shape of data larger than threshold: ', np.where(stc_fsave_all_real_diff_avg >thresh)[0].shape)
# zero out the values below threshold
for i in range(stc_fsave_all_real_diff_avg.shape[0]):
    if stc_fsave_all_real_diff_avg[i] < thresh:
        stc_fsave_all_real_diff_avg[i] = 0

stc_diff = prepareInverseSolution_group(stc_fsave_all_real_diff_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)

#print(np.max(stc_diff.data))
# Visualize the thresholded data
showResult('fsaverage', sourceFolder, stc_diff, 
           '2real_change_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh), subjects_dir,
           minimum=thresh, maximum=1100)

